In [2]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers as L
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
import librosa
import numpy as np
from tensorflow.keras.models import  model_from_json
class Voice_confidence:
    def __init__(self) -> None:
        self.model_weights = 'CNN_model_weights.weights.h5'
        self.model_encoder = 'encoder2.pickle'
        self.model_scaler = 'scaler2.pickle'
        self.model_json = 'CNN_model.json'

        json_file = open(self.model_json, 'r')
        loaded_model_json = json_file.read()
 
        self.model = model_from_json(loaded_model_json)
        
        # Load the model weights
        self.model.load_weights(self.model_weights)

    def load_pickle_file(self, pickle_path):
        with open(pickle_path, 'rb') as f:
            return pickle.load(f)

    def zcr(self, data, frame_length, hop_length):
        zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
        return np.squeeze(zcr)

    def rmse(self, data, frame_length=2048, hop_length=512):
        rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
        return np.squeeze(rmse)

    def mfcc(self, data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
        mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=13, hop_length=hop_length)
        return np.squeeze(mfcc.T) if not flatten else np.ravel(mfcc.T)

    def extract_features(self, data, sr=22050, frame_length=2048, hop_length=512):
        # Extract individual features
        zcr_result = self.zcr(data, frame_length, hop_length)
        rmse_result = self.rmse(data, frame_length, hop_length)
        mfcc_result = self.mfcc(data, sr, frame_length, hop_length, flatten=True)

        # Concatenate features
        combined_features = np.hstack((zcr_result, rmse_result, mfcc_result))

        # Check total size
        total_features = len(combined_features)

        # Pad or truncate to ensure exactly 1620 features
        if total_features < 1620:
            combined_features = np.pad(combined_features, (0, 1620 - total_features), mode='constant')
        elif total_features > 1620:
            combined_features = combined_features[:1620]

        # Reshape to (1, 1620)
        combined_features = np.reshape(combined_features, newshape=(1, 1620))

        return combined_features

    def et_predict_feat(self, file_path):
        data, sr = librosa.load(file_path, sr=None)
        features = self.extract_features(data, sr)
        scaler = self.load_pickle_file(self.model_scaler)
        scaled_features = scaler.transform(features)
        return scaled_features

    def prediction(self, path1):
        res = self.et_predict_feat(path1)
        res = res.reshape(res.shape[0], res.shape[1], 1)
        # Make predictions
        predictions = self.model.predict(res)
     
    
        encoder2 = self.load_pickle_file(self.model_encoder)
        y_pred = encoder2.inverse_transform(predictions)
        return y_pred[0]

if __name__ == '__main__':
    voice = Voice_confidence()
    print(voice.prediction('1001_DFA_ANG_XX.wav'))


/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


['angry']


/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


segmentation for the clips

In [34]:
data,sr = librosa.load("sample")
segment_samples = int(2.5 * sr)

/tmp/ipykernel_9906/727660529.py:1: UserWarning: PySoundFile failed. Trying audioread instead.
  data,sr = librosa.load("WhatsApp Audio 2024-07-11 at 12.37.05 PM.mp4")
/home/lang_chain/anaconda3/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [36]:
total_samples = len(data)
segment_samples,total_samples

(55125, 221088)

In [37]:
for start in range(0, total_samples, segment_samples):
    end = start + segment_samples
    print(end)

55125
110250
165375
220500
275625


{1: 'Neutral', 2: 'Calm', 3: 'Happy', 4: 'Sad', 5: 'Angry', 6: 'Fear', 7: 'Disgust', 8: 'Surprise'}
        
data = {1: 'Calm', 2: 'Calm', 3: 'Confident', 4: 'Under confident', 5: 'Confident', 6: 'under confident', 7: 'under confident', 8: 'Confiddent'}

In [50]:
import pickle
import librosa
import numpy as np
from tensorflow.keras.models import model_from_json

class Voice_confidence:
    def __init__(self):
        self.model_weights = 'CNN_model_weights.weights.h5'
        self.model_encoder = 'encoder2.pickle'
        self.model_scaler = 'scaler2.pickle'
        self.model_json = 'CNN_model.json'

        self.emotion_mapping = {
            'neutral': 'Calm',
            'calm': 'Calm',
            'happy': 'Confident',
            'sad': 'Under confident',
            'angry': 'Confident',
            'fear': 'Under confident',
            'disgust': 'Under confident',
            'surprise': 'Confident'
        }
        # Load model architecture from JSON
        json_file = open(self.model_json, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        
        # Load the model weights
        self.model.load_weights(self.model_weights)

    def load_pickle_file(self, pickle_path):
        with open(pickle_path, 'rb') as f:
            return pickle.load(f)

    def zcr(self, data, frame_length, hop_length):
        zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
        return np.squeeze(zcr)

    def rmse(self, data, frame_length=2048, hop_length=512):
        rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
        return np.squeeze(rmse)

    def mfcc(self, data, sr, frame_length=2048, hop_length=512, flatten=True):
        mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=13, hop_length=hop_length)
        return np.squeeze(mfcc.T) if not flatten else np.ravel(mfcc.T)

    def extract_features(self, data, sr=22050, frame_length=2048, hop_length=512):
        zcr_result = self.zcr(data, frame_length, hop_length)
        rmse_result = self.rmse(data, frame_length, hop_length)
        mfcc_result = self.mfcc(data, sr, frame_length, hop_length, flatten=True)
        
        combined_features = np.hstack((zcr_result, rmse_result, mfcc_result))
        total_features = len(combined_features)
        
        if total_features < 1620:
            combined_features = np.pad(combined_features, (0, 1620 - total_features), mode='constant')
        elif total_features > 1620:
            combined_features = combined_features[:1620]
        
        combined_features = np.reshape(combined_features, newshape=(1, 1620))
        return combined_features

    def et_predict_feat(self, data):
        features = self.extract_features(data)
        scaler = self.load_pickle_file(self.model_scaler)
        scaled_features = scaler.transform(features)
        return scaled_features

    def prediction(self, data):
        res = self.et_predict_feat(data)
        res = res.reshape(res.shape[0], res.shape[1], 1)
        predictions = self.model.predict(res)
        encoder2 = self.load_pickle_file(self.model_encoder)
        y_pred = encoder2.inverse_transform(predictions)

        # Debugging prints
        print("y_pred values:", y_pred)

        predicted_label = y_pred[0][0] 
        if predicted_label in self.emotion_mapping:
            original_emotion = self.emotion_mapping[predicted_label]

            return original_emotion
        else:
            return "Unknown Emotion"

    def process_audio_file(self, audio_file, clip_duration=2.5):
        data, sr = librosa.load(audio_file, sr=None) 
        clip_length = int(clip_duration * sr)
        predictions = []

        # Segment the audio into clips
        for i in range(0, len(data), clip_length):
            segment = data[i:i+clip_length]
            if len(segment) < clip_length:
                # Pad the segment if it's shorter than the desired clip length
                segment = np.pad(segment, (0, clip_length - len(segment)), mode='constant')

            # Perform prediction on the segment
            prediction = self.prediction(segment)
            predictions.append(prediction)

        return predictions

if __name__ == '__main__':
    voice = Voice_confidence()
    audio_file = 'sample'  
    predictions = voice.process_audio_file(audio_file)
    print(predictions)


/tmp/ipykernel_9906/3311293452.py:89: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sr = librosa.load(audio_file, sr=None)
/home/lang_chain/anaconda3/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
y_pred values: [['fear']]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
y_pred values: [['sad']]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
y_pred values: [['sad']]


/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when us

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
y_pred values: [['sad']]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
y_pred values: [['fear']]
['Under confident', 'Under confident', 'Under confident', 'Under confident', 'Under confident']


/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/lang_chain/anaconda3/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.2 when us